<a href="https://colab.research.google.com/github/luis0098/Datos-Mundial-Qatar/blob/main/Prediccion_Mundial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Librerias

In [ ]:
import pandas as pd
import pickle
from scipy.stats import poisson

Lectura de Datos

In [ ]:
#Lectura de tabla en formato Diccionario de los Grupos 
dict_table = pickle.load(open('dict_table','rb'))
#Lectura de datos historicos de goles
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
#Lectura de datos de fixture 
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

Grupos

In [ ]:
dict_table["Group A"]   #Group B   Group C    Group D  .... 

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


# FUERZA DE UN EQUIPO

Creacion de un DataFrame que me indique el "poder" que tiene un equipo para anotar un gol y qué tan "debil" es un equipo en recibir goles, segun la data historica de goles anotados a un equipo contrario en un partido y goles recibidos del equipo contrario en un partido.

In [ ]:
#Ordenando DataFrames
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]
df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})
#calculo del promedio de goles anotados por partido y promedio de goles que le anotaron por partido
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()

Data Historica

In [ ]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,France,Peru,2018,1,0,1


Cabe resalta que la condicion de "Local" y "Visitante" solo es un etiqueta de nombre, debido a que este parametro no afecta en un mundial debido a que todos los paises juegan los partidos de Local en Qatar. El parametro de "Local" y "Visitante" afecta en los partidos desarrollados en Ligas.

In [ ]:
#DataFrame de la Fuerza de un Equipo
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


Observamos que Argentina tiene más probabilidad de anotar un gol a que le anoten un gol.
Austria tiene una probabilidad parecidad a Argentina de anotar un gol, pero cuenta con mayor probabilidad de que le anoten un gol.
En un escenario de Argentina vs Austria... Austria perderia ante Argentina debido a que Argentina cuenta con una mayor probabilidad de anotar un gol y Austria cuenta con una mayor probabilidad de que le anoten un gol.
Una posible desventaja de este analisis es que la probabilidad que tienen los equipos depende de cuantos goles ha metido en un partido y esto es un dato muy dependiente de las circunstancias del escenario.

# FUNCION PREDICCION DE PUNTAJES

**Usaremos la Distribucion de Poisson**


La distribucion Poisson es una distribucion discreta que describe el numero de eventos que ocurren en un intervalo de tiempo fijo o region de oportunidad. Definiremos al *GOL* como un evento que puede ocurrir en los 90 minutos de un partido de futbol

Condiciones para considerar la Distribucion de Poisson

1. El numero de Eventos se puede contar

2. La ocurrencia de Eventos son independientes

3. La tasa a la que ocurren los eventos son constantes

4. Dos eventos no puede ocurrir exactamente en el mismo instante de tiempo
    
**Se cumple la Condicion 1**: El evento con el que estamos trabajando, los goles anotados, se pueden contar. 

**Se cumple la Condicion 2**: Estamos considerando que la ocurrencia de un gol no afecta la ocurrencia de otro gol en el transcurso del partido. En la practica esto no se cumple debido a que la ocurrencia de un gol puede motivar al equipo que lo anoto y desalentar al equipo al que le anotaron, pero nosotros consideraremos que la ocurrencia de un gol no afectará de que otro gol ocurra.

**Se cumple la Condicion 3**: Esta condicion está relacionada a que la ocurrencia del evento en un intervalo de tiempo debe ser la misma probabilidad de que ocurra en otro intervalo de tiempo y nuestro evento cumple con esa condicion debido a que la probabilidad de que ocurra un gol al enfrentarse dos equipos en un partido de 90 minutos es la misma a que esos dos equipos se enfrenten en otro partido de 90 minutos

**Se cumple la Condicion 4**: Dos goles no pueden ser anotados en el mismo instante de tiempo


Existe mucho más parametros pero por tratarse de un caso practico no los vamos a considerar

In [ ]:
#Funcion prediccion
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored']*df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored']*df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

# FUNCION DE PRUEBA

Observamos que en un partido Argentinca vs Mexico, Argentinca tiene una probabilidad de ganar de 2.3129 mientras que Mexico tiene una probabilidad de 0.5378 y nosotros consideraremos que ganará Argentina.


In [ ]:
print(predict_points('Argentina', 'Mexico'))

(2.3129151525530505, 0.5378377125059863)


Observamos que al no contar con data historica de Qatar no podemos predecir cómo será su desempeño contra Ecuador y en general, si no tenemos datos historicos de un equipo participando en un mundial vamos a considerar que no podremos predecir como será el resultado.

Por lo que consideraremos que Qatar no va a pasar la fase de grupos del Mundial.

In [ ]:
print(predict_points('Qatar (H)', 'Ecuador')) # Qatar vs Team X -> 0 puntos para ambos

(0, 0)


**Actualizado**:
El modelo no puede predecir si Ecuador le iba a ganar a Qatar, el partido de Qatar vs Ecuador que se realizó hace unos dias dio como ganador a Ecuador.
Nuestro modelo predice que Qatar no pasará la fase de grupos

# PREDICCION DEL MUNDIAL

## FASE DE GRUPOS

In [ ]:
#creamos los dataframe de la Fase de grupos:
df_fixture_group_48 = df_fixture[:48].copy()
#creamos los dataframe de los Octavos de finales:
df_fixture_knockout = df_fixture[48:56].copy()
#creamos los dataframe de los Cuartos de final:
df_fixture_quarter = df_fixture[56:60].copy()
#creamos los dataframe de la Semifinal:
df_fixture_semi = df_fixture[60:62].copy()
#creamos los dataframe de la Final:
df_fixture_final = df_fixture[62:].copy()

In [ ]:
#Corremos todos los partidos de la fase de frupos y actualizamos las tablas de cada grupo
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

Resultado del puntaje por cada Grupo en la Fase de Grupos

In [ ]:
dict_table['Group A']

,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


# OCTAVOS DE FINAL

Creamos los escenarios de los partidos que se jugaran en Octavos de Final

In [ ]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


Predecimos los ganadores de los Octavos de Final

In [ ]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


Observamos que nuestro modelo predice que Brasil le gana a Uruguay

# CUARTOS DE FINAL

Actualizamos la tabla para considerar los equipos que pasaron a Cuartos de Final

In [ ]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

Creamos los escenarios de los partidos que se jugaran en Cuartos de Final

In [ ]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


Predecimos los ganadores de Cuartos de Final

In [ ]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


Observamos que Paises Bajos se enfrenta contra Argentina y nuestro modelo predice que gana Paises Bajos.

Teniendo como cuarto finalistas a Brasil, Paises Bajos, Portugal y Francia

# SEMIFINAL

Actualizamos los partidos y creamos el Escenario

In [ ]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


Predecimos a los ganadores de la Semifinal

In [ ]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


Observamos que Francia le gana a Portugal y que Brasil le gana a Paises Bajos.

Quedando la Final Brasil vs Francia

# FINAL

Generamos los escenarios

In [ ]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [ ]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil


Teniendo como Ganador del Mundial Qatar 2022 a Brasil en la final contra Francia